# About this Notebook:

This notebook detailed the exploratory data analysis on the Open Images Classes. So we can prepared the ourself to how to use the data.

Notebook Enviroment

- Kernel: Python 3
- 

In [1]:
!python --version

Python 3.6.3 :: Anaconda custom (64-bit)


## Question to be answered:

1. How many classes can we use?
2. How many images perclass?

## TODO

# Load Modules

In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import json
import os
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

import seaborn as sns



# Commonly Shared Statics

In [3]:
data_labels_description_dir_path = '../../data/doc/labels/'

In [4]:
data_dir_path = '../../data/'

# EDA on the class-descriptions.csv

In [5]:
class_descriptions_file_path = data_labels_description_dir_path + 'class-descriptions.csv'


In [6]:
df_class_descriptions = pd.read_csv(class_descriptions_file_path,names=['LabelName', 'Description'])

In [7]:
df_class_descriptions.head(10)

,LabelName,Description
0,/m/0100nhbf,Sprenger's tulip
1,/m/0104x9kv,Vinegret
2,/m/0105jzwx,Dabu-dabu
3,/m/0105ld7g,Pistachio ice cream
4,/m/0105lxy5,Woku
5,/m/0105n86x,Pastila
6,/m/0105ts35,Burasa
7,/m/0108_09c,Summer snowflake
8,/m/01_097,Airmail
9,/m/010dmf,Isle of man tt


In [8]:
df_class_descriptions.tail(10)

,LabelName,Description
19985,/m/0zdntfj,Candy cane sorrel
19986,/m/0zg7256,Gingerbread house
19987,/m/0zghq,Moonshine
19988,/m/0zkm,Arthropod
19989,/m/0zlm,Conservation-restoration
19990,/m/0zrpfhj,Throwing
19991,/m/0zrrls2,Red dahlia
19992,/m/0zrthkd,Brine
19993,/m/0zrv5th,Lamborghini huracán
19994,/m/0zvk5,Helmet


In [9]:
df_class_descriptions.describe()

,LabelName,Description
count,19995,19995
unique,19995,19712
top,/m/02qhfw3,Trunk
freq,1,4


In [10]:
df_class_descriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 2 columns):
LabelName      19995 non-null object
Description    19995 non-null object
dtypes: object(2)
memory usage: 312.5+ KB


Replace space with underscore for S3 bucket prefix 

In [11]:
df_class_descriptions.Description = df_class_descriptions.Description.str.replace(' ', '_')

In [12]:
df_class_descriptions.tail(10)

,LabelName,Description
19985,/m/0zdntfj,Candy_cane_sorrel
19986,/m/0zg7256,Gingerbread_house
19987,/m/0zghq,Moonshine
19988,/m/0zkm,Arthropod
19989,/m/0zlm,Conservation-restoration
19990,/m/0zrpfhj,Throwing
19991,/m/0zrrls2,Red_dahlia
19992,/m/0zrthkd,Brine
19993,/m/0zrv5th,Lamborghini_huracán
19994,/m/0zvk5,Helmet


## Compose the list of 600 original labels 

In [32]:
! ls ../../data/doc/labels/



bbox_labels_600_hierarchy.json classes-trainable.txt
class-descriptions.csv         classes.txt


In [33]:
bbox_labels_600_hierarchy_json_file_path = data_labels_description_dir_path + 'bbox_labels_600_hierarchy.json'

In [34]:
bbox_labels_600_json_data=open(bbox_labels_600_hierarchy_json_file_path).read()

In [35]:
bbox_labels_600_json = json.loads(bbox_labels_600_json_data)

In [36]:
bbox_labels_600_json

{'LabelName': '/m/0bl9f',
 'Subcategory': [{'LabelName': '/m/0242l'},
  {'LabelName': '/m/03120'},
  {'LabelName': '/m/0h8l4fh'},
  {'LabelName': '/m/0138tl',
   'Subcategory': [{'LabelName': '/m/0167gd'},
    {'LabelName': '/m/01j51'},
    {'LabelName': '/m/029b3'},
    {'LabelName': '/m/02wmf'},
    {'LabelName': '/m/02zt3'},
    {'LabelName': '/m/0kmg4'}]},
  {'LabelName': '/m/019dx1',
   'Subcategory': [{'LabelName': '/m/0174k2'},
    {'LabelName': '/m/01k6s3'},
    {'LabelName': '/m/029bxz'},
    {'LabelName': '/m/02pjr4'},
    {'LabelName': '/m/02wv84t'},
    {'LabelName': '/m/02x984l'},
    {'LabelName': '/m/03qhv5'},
    {'LabelName': '/m/03s_tn'},
    {'LabelName': '/m/03wvsk'},
    {'LabelName': '/m/040b_t'},
    {'LabelName': '/m/04169hn'},
    {'LabelName': '/m/04lvq_'},
    {'LabelName': '/m/063rgb'},
    {'LabelName': '/m/07xyvk'},
    {'LabelName': '/m/0fx9l'},
    {'LabelName': '/m/0ky7b'},
    {'LabelName': '/m/0llzx'},
    {'LabelName': '/m/0c3m8g'},
    {'LabelName':

In [46]:
def item_generator(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                for child_val in item_generator(v, lookup_key):
                    yield child_val
    elif isinstance(json_input, list):
        for item in json_input:
            for item_val in item_generator(item, lookup_key):
                yield item_val

In [47]:
subcategory_set = set()
for label in item_generator(bbox_labels_600_json, "LabelName"):  
    subcategory_set.add(label)

In [48]:
subcategory_list =list(subcategory_set)

In [49]:
df_bbox_600_labels = pd.DataFrame(subcategory_list,columns=['LabelName'])

In [50]:
df_bbox_600_labels.head()

,LabelName
0,/m/057p5t
1,/m/0k5j
2,/m/09d5_
3,/m/01m4t
4,/m/04zpv


In [51]:
df_bbox_600_labels.describe()

,LabelName
count,602
unique,602
top,/m/02p5f1q
freq,1


## Filter out 600 classes from class descriptions

In [52]:
df_trainables = pd.merge(df_bbox_600_labels,
                 df_class_descriptions[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [53]:
df_trainables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601 entries, 0 to 600
Data columns (total 2 columns):
LabelName      601 non-null object
Description    601 non-null object
dtypes: object(2)
memory usage: 14.1+ KB


In [54]:
df_trainables.head(10)

,LabelName,Description
0,/m/057p5t,Scoreboard
1,/m/0k5j,Aircraft
2,/m/09d5_,Owl
3,/m/01m4t,Printer
4,/m/04zpv,Milk
5,/m/06_72j,Digital_clock
6,/m/02xb7qb,Face_powder
7,/m/03nfch,Sandal
8,/m/081qc,Wine
9,/m/0f9_l,Snail


In [55]:
df_trainables.to_csv( data_dir_path + '600_trainable_LabelNames.csv', sep=',')

Listing all the labels 

In [56]:
for i in list(df_trainables['Description']):
    print (i)

Scoreboard
Aircraft
Owl
Printer
Milk
Digital_clock
Face_powder
Sandal
Wine
Snail
Lifejacket
Asparagus
Chainsaw
Laptop
Blender
Kitchen_&_dining_room_table
Cattle
Candy
Hammer
Table_tennis_racket
Human_mouth
Goggles
Squid
Countertop
Human_leg
Filing_cabinet
Organ
Hair_spray
Dinosaur
Cream
Lemon
Container
Ski
Nightstand
Cucumber
Pig
Hair_dryer
Eagle
Stethoscope
Sushi
Lavender
Bicycle_wheel
Violin
Mug
Training_bench
Waffle_iron
Sea_turtle
Canoe
Serving_tray
Tree_house
Poster
Squirrel
Toilet_paper
Turkey
Stool
Oyster
Coat
Cabbage
Truck
Skyscraper
Bronze_sculpture
Slow_cooker
Plant
Beer
Beetle
Surfboard
Teapot
Human_body
Bidet
Dolphin
Toy
Luggage_and_bags
Sea_lion
Shower
Cupboard
Tripod
Panda
Pitcher
Earrings
Human_ear
Tart
Armadillo
Shirt
Billiard_table
Popcorn
Infant_bed
Desk
Humidifier
Roller_skates
Table
Football_helmet
Watermelon
Cantaloupe
Horse
Wood-burning_stove
Axe
Scarf
Sandwich
Cello
Shotgun
Kitchen_utensil
Cookie
Shellfish
Computer_keyboard
Bowling_equipment
Skull
Drill
Koala
Mot

### Create a dictionary from from the df_trainables

In [57]:
trainables_dict = df_trainables.set_index("LabelName").to_dict(orient="index")

In [58]:
trainables_dict

{'/m/011k07': {'Description': 'Tortoise'},
 '/m/011q46kg': {'Description': 'Container'},
 '/m/012074': {'Description': 'Magpie'},
 '/m/0120dh': {'Description': 'Sea_turtle'},
 '/m/01226z': {'Description': 'Football'},
 '/m/012n7d': {'Description': 'Ambulance'},
 '/m/012w5l': {'Description': 'Ladder'},
 '/m/012xff': {'Description': 'Toothbrush'},
 '/m/012ysf': {'Description': 'Syringe'},
 '/m/0130jx': {'Description': 'Sink'},
 '/m/0138tl': {'Description': 'Toy'},
 '/m/013y1f': {'Description': 'Organ'},
 '/m/01432t': {'Description': 'Cassette_deck'},
 '/m/014j1m': {'Description': 'Apple'},
 '/m/014sv8': {'Description': 'Human_eye'},
 '/m/014trl': {'Description': 'Cosmetics'},
 '/m/014y4n': {'Description': 'Paddle'},
 '/m/0152hh': {'Description': 'Snowman'},
 '/m/01599': {'Description': 'Beer'},
 '/m/015h_t': {'Description': 'Human_beard'},
 '/m/015p6': {'Description': 'Bird'},
 '/m/015qbp': {'Description': 'Parking_meter'},
 '/m/015qff': {'Description': 'Traffic_light'},
 '/m/015wgc': {'

In [61]:
for key, value in trainables_dict.items():
    print(value['Description'])

Scoreboard
Aircraft
Owl
Printer
Milk
Digital_clock
Face_powder
Sandal
Wine
Snail
Lifejacket
Asparagus
Chainsaw
Laptop
Blender
Kitchen_&_dining_room_table
Cattle
Candy
Hammer
Table_tennis_racket
Human_mouth
Goggles
Squid
Countertop
Human_leg
Filing_cabinet
Organ
Hair_spray
Dinosaur
Cream
Lemon
Container
Ski
Nightstand
Cucumber
Pig
Hair_dryer
Eagle
Stethoscope
Sushi
Lavender
Bicycle_wheel
Violin
Mug
Training_bench
Waffle_iron
Sea_turtle
Canoe
Serving_tray
Tree_house
Poster
Squirrel
Toilet_paper
Turkey
Stool
Oyster
Coat
Cabbage
Truck
Skyscraper
Bronze_sculpture
Slow_cooker
Plant
Beer
Beetle
Surfboard
Teapot
Human_body
Bidet
Dolphin
Toy
Luggage_and_bags
Sea_lion
Shower
Cupboard
Tripod
Panda
Pitcher
Earrings
Human_ear
Tart
Armadillo
Shirt
Billiard_table
Popcorn
Infant_bed
Desk
Humidifier
Roller_skates
Table
Football_helmet
Watermelon
Cantaloupe
Horse
Wood-burning_stove
Axe
Scarf
Sandwich
Cello
Shotgun
Kitchen_utensil
Cookie
Shellfish
Computer_keyboard
Bowling_equipment
Skull
Drill
Koala
Mot

Adding label '/m/0bl9f' with description "Entity"  

In [62]:
trainables_dict['/m/0bl9f'] = {'Description': 'Entity'}

In [63]:
trainables_dict['/m/0bl9f']

{'Description': 'Entity'}

In [64]:
trainables_dict['/m/0152hh']

{'Description': 'Snowman'}

## Convert labels to human readable description

In [65]:
bbox_labels_600_json = json.loads(bbox_labels_600_json_data)

In [67]:
def label_to_description(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                json_input[k] = trainables_dict[v]['Description']
 
                yield v
            else:
                for child_val in label_to_description(v, lookup_key):
                      
                    yield child_val
    elif isinstance(json_input, list):
        for item in json_input:
            for item_val in label_to_description(item, lookup_key):
                yield item_val

In [68]:
for label in label_to_description(bbox_labels_600_json, "LabelName"):  
    print("")

# label_to_description(bbox_labels_600_json, "LabelName")

In [69]:
bbox_labels_600_json

{'LabelName': 'Entity',
 'Subcategory': [{'LabelName': 'Coin'},
  {'LabelName': 'Flag'},
  {'LabelName': 'Light_bulb'},
  {'LabelName': 'Toy',
   'Subcategory': [{'LabelName': 'Doll'},
    {'LabelName': 'Balloon'},
    {'LabelName': 'Dice'},
    {'LabelName': 'Flying_disc'},
    {'LabelName': 'Kite'},
    {'LabelName': 'Teddy_bear'}]},
  {'LabelName': 'Home_appliance',
   'Subcategory': [{'LabelName': 'Washing_machine'},
    {'LabelName': 'Toaster'},
    {'LabelName': 'Oven'},
    {'LabelName': 'Blender'},
    {'LabelName': 'Gas_stove'},
    {'LabelName': 'Mechanical_fan'},
    {'LabelName': 'Heater'},
    {'LabelName': 'Kettle'},
    {'LabelName': 'Hair_dryer'},
    {'LabelName': 'Refrigerator'},
    {'LabelName': 'Wood-burning_stove'},
    {'LabelName': 'Humidifier'},
    {'LabelName': 'Mixer'},
    {'LabelName': 'Coffeemaker'},
    {'LabelName': 'Microwave_oven'},
    {'LabelName': 'Dishwasher'},
    {'LabelName': 'Sewing_machine'},
    {'LabelName': 'Hand_dryer'},
    {'LabelName':

## EDA on the hierchy of 600 original labels by classes¶

In [71]:
bbox_labels_600_json

{'LabelName': 'Entity',
 'Subcategory': [{'LabelName': 'Coin'},
  {'LabelName': 'Flag'},
  {'LabelName': 'Light_bulb'},
  {'LabelName': 'Toy',
   'Subcategory': [{'LabelName': 'Doll'},
    {'LabelName': 'Balloon'},
    {'LabelName': 'Dice'},
    {'LabelName': 'Flying_disc'},
    {'LabelName': 'Kite'},
    {'LabelName': 'Teddy_bear'}]},
  {'LabelName': 'Home_appliance',
   'Subcategory': [{'LabelName': 'Washing_machine'},
    {'LabelName': 'Toaster'},
    {'LabelName': 'Oven'},
    {'LabelName': 'Blender'},
    {'LabelName': 'Gas_stove'},
    {'LabelName': 'Mechanical_fan'},
    {'LabelName': 'Heater'},
    {'LabelName': 'Kettle'},
    {'LabelName': 'Hair_dryer'},
    {'LabelName': 'Refrigerator'},
    {'LabelName': 'Wood-burning_stove'},
    {'LabelName': 'Humidifier'},
    {'LabelName': 'Mixer'},
    {'LabelName': 'Coffeemaker'},
    {'LabelName': 'Microwave_oven'},
    {'LabelName': 'Dishwasher'},
    {'LabelName': 'Sewing_machine'},
    {'LabelName': 'Hand_dryer'},
    {'LabelName':

In [129]:
def hierchy_generator(json_input, lookup_key, parent_key):
    if isinstance(json_input, dict):
        
        new_parent_key = json_input[lookup_key]
        
        print("new_parent_key: ",new_parent_key )
        
        
        for k, v in json_input.items():
 
            if k == lookup_key:
                print("added new key")
                classes_hierchy_dict[v] = {}
                yield v
                
            else:
                print("else new_parent_key: ",new_parent_key )
                
                for child_val in hierchy_generator(v, lookup_key, new_parent_key):
                    
                    print("else child_val:", child_val)
#                     print("else v: " , v)


                    yield child_val
    elif isinstance(json_input, list):
        
        print("elif new_parent_key: ",parent_key )
        for item in json_input:
            print("item: ", item)
            for item_val in hierchy_generator(item, lookup_key, parent_key):
                print("item_val: ", item_val)
                classes_hierchy_dict[item_val]['parent'] = parent_key
                
#                 classes_hierchy_dict[parent_key]['children'].append(item_val)
                yield item_val

In [130]:
classes_hierchy_dict = {}

for label in hierchy_generator(bbox_labels_600_json, "LabelName", None):  
    print("Do nothing")

new_parent_key:  Entity
added new key
Do nothing
else new_parent_key:  Entity
elif new_parent_key:  Entity
item:  {'LabelName': 'Coin'}
new_parent_key:  Coin
added new key
item_val:  Coin
else child_val: Coin
Do nothing
item:  {'LabelName': 'Flag'}
new_parent_key:  Flag
added new key
item_val:  Flag
else child_val: Flag
Do nothing
item:  {'LabelName': 'Light_bulb'}
new_parent_key:  Light_bulb
added new key
item_val:  Light_bulb
else child_val: Light_bulb
Do nothing
item:  {'LabelName': 'Toy', 'Subcategory': [{'LabelName': 'Doll'}, {'LabelName': 'Balloon'}, {'LabelName': 'Dice'}, {'LabelName': 'Flying_disc'}, {'LabelName': 'Kite'}, {'LabelName': 'Teddy_bear'}]}
new_parent_key:  Toy
added new key
item_val:  Toy
else child_val: Toy
Do nothing
else new_parent_key:  Toy
elif new_parent_key:  Toy
item:  {'LabelName': 'Doll'}
new_parent_key:  Doll
added new key
item_val:  Doll
else child_val: Doll
item_val:  Doll
else child_val: Doll
Do nothing
item:  {'LabelName': 'Balloon'}
new_parent_key: 

new_parent_key:  Pencil_case
added new key
item_val:  Pencil_case
else child_val: Pencil_case
item_val:  Pencil_case
else child_val: Pencil_case
Do nothing
item:  {'LabelName': 'Plastic_bag'}
new_parent_key:  Plastic_bag
added new key
item_val:  Plastic_bag
else child_val: Plastic_bag
item_val:  Plastic_bag
else child_val: Plastic_bag
Do nothing
item:  {'LabelName': 'Paper_cutter'}
new_parent_key:  Paper_cutter
added new key
item_val:  Paper_cutter
else child_val: Paper_cutter
item_val:  Paper_cutter
else child_val: Paper_cutter
Do nothing
item:  {'LabelName': 'Toilet_paper'}
new_parent_key:  Toilet_paper
added new key
item_val:  Toilet_paper
else child_val: Toilet_paper
item_val:  Toilet_paper
else child_val: Toilet_paper
Do nothing
item:  {'LabelName': 'Envelope'}
new_parent_key:  Envelope
added new key
item_val:  Envelope
else child_val: Envelope
item_val:  Envelope
else child_val: Envelope
Do nothing
item:  {'LabelName': 'Pen'}
new_parent_key:  Pen
added new key
item_val:  Pen
else

item_val:  Window_blind
else child_val: Window_blind
Do nothing
item:  {'LabelName': 'Curtain'}
new_parent_key:  Curtain
added new key
item_val:  Curtain
else child_val: Curtain
Do nothing
item:  {'LabelName': 'Mirror'}
new_parent_key:  Mirror
added new key
item_val:  Mirror
else child_val: Mirror
Do nothing
item:  {'LabelName': 'Sculpture', 'Subcategory': [{'LabelName': 'Snowman'}, {'LabelName': 'Bust'}, {'LabelName': 'Bronze_sculpture'}]}
new_parent_key:  Sculpture
added new key
item_val:  Sculpture
else child_val: Sculpture
Do nothing
else new_parent_key:  Sculpture
elif new_parent_key:  Sculpture
item:  {'LabelName': 'Snowman'}
new_parent_key:  Snowman
added new key
item_val:  Snowman
else child_val: Snowman
item_val:  Snowman
else child_val: Snowman
Do nothing
item:  {'LabelName': 'Bust'}
new_parent_key:  Bust
added new key
item_val:  Bust
else child_val: Bust
item_val:  Bust
else child_val: Bust
Do nothing
item:  {'LabelName': 'Bronze_sculpture'}
new_parent_key:  Bronze_sculpture

item_val:  Helmet
else child_val: Helmet
item_val:  Helmet
else child_val: Helmet
item_val:  Helmet
else child_val: Helmet
Do nothing
else new_parent_key:  Helmet
elif new_parent_key:  Helmet
item:  {'LabelName': 'Bicycle_helmet'}
new_parent_key:  Bicycle_helmet
added new key
item_val:  Bicycle_helmet
else child_val: Bicycle_helmet
item_val:  Bicycle_helmet
else child_val: Bicycle_helmet
item_val:  Bicycle_helmet
else child_val: Bicycle_helmet
item_val:  Bicycle_helmet
else child_val: Bicycle_helmet
Do nothing
item:  {'LabelName': 'Football_helmet'}
new_parent_key:  Football_helmet
added new key
item_val:  Football_helmet
else child_val: Football_helmet
item_val:  Football_helmet
else child_val: Football_helmet
item_val:  Football_helmet
else child_val: Football_helmet
item_val:  Football_helmet
else child_val: Football_helmet
Do nothing
item:  {'LabelName': 'Billiard_table'}
new_parent_key:  Billiard_table
added new key
item_val:  Billiard_table
else child_val: Billiard_table
item_val

item:  {'LabelName': 'Tower'}
new_parent_key:  Tower
added new key
item_val:  Tower
else child_val: Tower
item_val:  Tower
else child_val: Tower
Do nothing
item:  {'LabelName': 'House'}
new_parent_key:  House
added new key
item_val:  House
else child_val: House
item_val:  House
else child_val: House
Do nothing
item:  {'LabelName': 'Office_building'}
new_parent_key:  Office_building
added new key
item_val:  Office_building
else child_val: Office_building
item_val:  Office_building
else child_val: Office_building
Do nothing
item:  {'LabelName': 'Convenience_store'}
new_parent_key:  Convenience_store
added new key
item_val:  Convenience_store
else child_val: Convenience_store
item_val:  Convenience_store
else child_val: Convenience_store
Do nothing
item:  {'LabelName': 'Swimming_pool'}
new_parent_key:  Swimming_pool
added new key
item_val:  Swimming_pool
else child_val: Swimming_pool
Do nothing
item:  {'LabelName': 'Person', 'Part': [{'LabelName': 'Human_body', 'Part': [{'LabelName': 'Hum

else child_val: Artichoke
Do nothing
item:  {'LabelName': 'Potato'}
new_parent_key:  Potato
added new key
item_val:  Potato
else child_val: Potato
item_val:  Potato
else child_val: Potato
item_val:  Potato
else child_val: Potato
Do nothing
item:  {'LabelName': 'Tomato'}
new_parent_key:  Tomato
added new key
item_val:  Tomato
else child_val: Tomato
item_val:  Tomato
else child_val: Tomato
item_val:  Tomato
else child_val: Tomato
Do nothing
item:  {'LabelName': 'Asparagus'}
new_parent_key:  Asparagus
added new key
item_val:  Asparagus
else child_val: Asparagus
item_val:  Asparagus
else child_val: Asparagus
item_val:  Asparagus
else child_val: Asparagus
Do nothing
item:  {'LabelName': 'Squash', 'Subcategory': [{'LabelName': 'Pumpkin'}, {'LabelName': 'Zucchini'}]}
new_parent_key:  Squash
added new key
item_val:  Squash
else child_val: Squash
item_val:  Squash
else child_val: Squash
item_val:  Squash
else child_val: Squash
Do nothing
else new_parent_key:  Squash
elif new_parent_key:  Squash

else child_val: Sun_hat
item_val:  Sun_hat
else child_val: Sun_hat
item_val:  Sun_hat
else child_val: Sun_hat
Do nothing
item:  {'LabelName': 'Scarf'}
new_parent_key:  Scarf
added new key
item_val:  Scarf
else child_val: Scarf
item_val:  Scarf
else child_val: Scarf
Do nothing
item:  {'LabelName': 'Skirt', 'Subcategory': [{'LabelName': 'Miniskirt'}]}
new_parent_key:  Skirt
added new key
item_val:  Skirt
else child_val: Skirt
item_val:  Skirt
else child_val: Skirt
Do nothing
else new_parent_key:  Skirt
elif new_parent_key:  Skirt
item:  {'LabelName': 'Miniskirt'}
new_parent_key:  Miniskirt
added new key
item_val:  Miniskirt
else child_val: Miniskirt
item_val:  Miniskirt
else child_val: Miniskirt
item_val:  Miniskirt
else child_val: Miniskirt
Do nothing
item:  {'LabelName': 'Jacket'}
new_parent_key:  Jacket
added new key
item_val:  Jacket
else child_val: Jacket
item_val:  Jacket
else child_val: Jacket
Do nothing
item:  {'LabelName': 'Fashion_accessory', 'Subcategory': [{'LabelName': 'Glov

else child_val: Caterpillar
item_val:  Caterpillar
else child_val: Caterpillar
item_val:  Caterpillar
else child_val: Caterpillar
item_val:  Caterpillar
else child_val: Caterpillar
item_val:  Caterpillar
else child_val: Caterpillar
Do nothing
item:  {'LabelName': 'Butterfly'}
new_parent_key:  Butterfly
added new key
item_val:  Butterfly
else child_val: Butterfly
item_val:  Butterfly
else child_val: Butterfly
item_val:  Butterfly
else child_val: Butterfly
item_val:  Butterfly
else child_val: Butterfly
item_val:  Butterfly
else child_val: Butterfly
Do nothing
item:  {'LabelName': 'Dragonfly'}
new_parent_key:  Dragonfly
added new key
item_val:  Dragonfly
else child_val: Dragonfly
item_val:  Dragonfly
else child_val: Dragonfly
item_val:  Dragonfly
else child_val: Dragonfly
item_val:  Dragonfly
else child_val: Dragonfly
Do nothing
item:  {'LabelName': 'Scorpion'}
new_parent_key:  Scorpion
added new key
item_val:  Scorpion
else child_val: Scorpion
item_val:  Scorpion
else child_val: Scorpion

item_val:  Shark
else child_val: Shark
item_val:  Shark
else child_val: Shark
Do nothing
item:  {'LabelName': 'Rays_and_skates'}
new_parent_key:  Rays_and_skates
added new key
item_val:  Rays_and_skates
else child_val: Rays_and_skates
item_val:  Rays_and_skates
else child_val: Rays_and_skates
item_val:  Rays_and_skates
else child_val: Rays_and_skates
Do nothing
item:  {'LabelName': 'Seahorse'}
new_parent_key:  Seahorse
added new key
item_val:  Seahorse
else child_val: Seahorse
item_val:  Seahorse
else child_val: Seahorse
item_val:  Seahorse
else child_val: Seahorse
Do nothing
item:  {'LabelName': 'Shellfish', 'Subcategory': [{'LabelName': 'Oyster'}, {'LabelName': 'Lobster'}, {'LabelName': 'Shrimp'}, {'LabelName': 'Crab'}]}
new_parent_key:  Shellfish
added new key
item_val:  Shellfish
else child_val: Shellfish
item_val:  Shellfish
else child_val: Shellfish
Do nothing
else new_parent_key:  Shellfish
elif new_parent_key:  Shellfish
item:  {'LabelName': 'Oyster'}
new_parent_key:  Oyster
ad

added new key
item_val:  Wrench
else child_val: Wrench
item_val:  Wrench
else child_val: Wrench
Do nothing
item:  {'LabelName': 'Flashlight'}
new_parent_key:  Flashlight
added new key
item_val:  Flashlight
else child_val: Flashlight
item_val:  Flashlight
else child_val: Flashlight
Do nothing
item:  {'LabelName': 'Scissors'}
new_parent_key:  Scissors
added new key
item_val:  Scissors
else child_val: Scissors
item_val:  Scissors
else child_val: Scissors
Do nothing
item:  {'LabelName': 'Ratchet'}
new_parent_key:  Ratchet
added new key
item_val:  Ratchet
else child_val: Ratchet
item_val:  Ratchet
else child_val: Ratchet
Do nothing
item:  {'LabelName': 'Kitchen_utensil', 'Subcategory': [{'LabelName': 'Chopsticks'}, {'LabelName': 'Ladle'}, {'LabelName': 'Spatula'}, {'LabelName': 'Can_opener'}, {'LabelName': 'Cutting_board'}, {'LabelName': 'Whisk'}, {'LabelName': 'Drinking_straw'}, {'LabelName': 'Knife'}, {'LabelName': 'Bottle_opener'}, {'LabelName': 'Measuring_cup'}, {'LabelName': 'Pizza_cut

In [128]:
classes_hierchy_dict

{'Accordion': {'parent': 'Entity'},
 'Adhesive_tape': {'parent': 'Entity'},
 'Aircraft': {'parent': 'Entity'},
 'Airplane': {'parent': 'Entity'},
 'Alarm_clock': {'parent': 'Entity'},
 'Alpaca': {'parent': 'Entity'},
 'Ambulance': {'parent': 'Entity'},
 'Animal': {'parent': 'Entity'},
 'Ant': {'parent': 'Entity'},
 'Antelope': {'parent': 'Entity'},
 'Apple': {'parent': 'Entity'},
 'Armadillo': {'parent': 'Entity'},
 'Artichoke': {'parent': 'Entity'},
 'Asparagus': {'parent': 'Entity'},
 'Auto_part': {'parent': 'Entity'},
 'Axe': {'parent': 'Entity'},
 'Backpack': {'parent': 'Entity'},
 'Bagel': {'parent': 'Entity'},
 'Baked_goods': {'parent': 'Entity'},
 'Balance_beam': {'parent': 'Entity'},
 'Ball': {'parent': 'Entity'},
 'Balloon': {'parent': 'Entity'},
 'Banana': {'parent': 'Entity'},
 'Band-aid': {'parent': 'Entity'},
 'Banjo': {'parent': 'Entity'},
 'Barge': {'parent': 'Entity'},
 'Barrel': {'parent': 'Entity'},
 'Baseball_bat': {'parent': 'Entity'},
 'Baseball_glove': {'parent': 

In [122]:
classes_hierchy_dict['Accordion']['parent']

'Entity'

## EDA on the classes-trainable.txt

In [117]:
classes_trainable_file_path = data_labels_description_dir_path + 'classes-trainable.txt'



In [118]:
df_classes_trainable = pd.read_csv(classes_trainable_file_path,names=['LabelName'])

In [119]:
df_classes_trainable.head(10)

,LabelName
0,/m/0108_09c
1,/m/010dmf
2,/m/010jjr
3,/m/010l12
4,/m/010lq47b
5,/m/010ls_cv
6,/m/01127
7,/m/01_12b
8,/m/0117_25k
9,/m/0117z


In [120]:
df_classes_trainable.describe()

,LabelName
count,7186
unique,7186
top,/m/02qg91d
freq,1


In [15]:
df_classes_trainable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7186 entries, 0 to 7185
Data columns (total 1 columns):
LabelName    7186 non-null object
dtypes: object(1)
memory usage: 112.3+ KB


## Filter out trainable classes from class descriptions

In [16]:
df_trainables = pd.merge(df_classes_trainable,
                 df_class_descriptions[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [17]:
df_trainables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7186 entries, 0 to 7185
Data columns (total 2 columns):
LabelName      7186 non-null object
Description    7186 non-null object
dtypes: object(2)
memory usage: 168.4+ KB


In [18]:
df_trainables.head()

,LabelName,Description
0,/m/0108_09c,Summer snowflake
1,/m/010dmf,Isle of man tt
2,/m/010jjr,Amusement park
3,/m/010l12,Roller coaster
4,/m/010lq47b,Witch hat


## EDA on train-annotations-human-imagelabels.csv

In [19]:
train_annotations_human_imagelabels_file_path = data_labels_description_dir_path + 'train-annotations-human-imagelabels.csv'

In [20]:
df_train_annotations_human_imagelabels = pd.read_csv(train_annotations_human_imagelabels_file_path,header=0)

In [21]:
df_train_annotations_human_imagelabels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27894289 entries, 0 to 27894288
Data columns (total 4 columns):
ImageID       object
Source        object
LabelName     object
Confidence    int64
dtypes: int64(1), object(3)
memory usage: 1.0+ GB


In [22]:
df_train_annotations_human_imagelabels.head()

,ImageID,Source,LabelName,Confidence
0,000002b66c9c498e,crowdsource-verification,/m/01kcnl,1
1,000002b66c9c498e,verification,/m/010l12,0
2,000002b66c9c498e,verification,/m/012c4n,0
3,000002b66c9c498e,verification,/m/012mj,1
4,000002b66c9c498e,verification,/m/012yh1,1


In [23]:
df_train_annotations_human_imagelabels_confident = df_train_annotations_human_imagelabels[df_train_annotations_human_imagelabels['Confidence']== 1
                                                                                         ]

In [24]:
df_train_annotations_human_imagelabels_confident.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13444569 entries, 0 to 27894288
Data columns (total 4 columns):
ImageID       object
Source        object
LabelName     object
Confidence    int64
dtypes: int64(1), object(3)
memory usage: 512.9+ MB


In [26]:
df_train_annotations_human_imagelabels_confident.head(20)

,ImageID,Source,LabelName,Confidence
0,000002b66c9c498e,crowdsource-verification,/m/01kcnl,1
3,000002b66c9c498e,verification,/m/012mj,1
4,000002b66c9c498e,verification,/m/012yh1,1
7,000002b66c9c498e,verification,/m/014sv8,1
14,000002b66c9c498e,verification,/m/016c68,1
16,000002b66c9c498e,verification,/m/016q19,1
23,000002b66c9c498e,verification,/m/019nj4,1
24,000002b66c9c498e,verification,/m/019_nn,1
25,000002b66c9c498e,verification,/m/019sc6,1
30,000002b66c9c498e,verification,/m/01bl7v,1


In [28]:
df_train_annotations_human_imagelabels_imgIDs_Labels = df_train_annotations_human_imagelabels[['ImageID','LabelName']]

In [29]:
df_train_annotations_human_imagelabels_imgIDs_Labels.head()

,ImageID,LabelName
0,000002b66c9c498e,/m/01kcnl
1,000002b66c9c498e,/m/010l12
2,000002b66c9c498e,/m/012c4n
3,000002b66c9c498e,/m/012mj
4,000002b66c9c498e,/m/012yh1


## Mergin with ImageIDs

In [38]:
df_trainables_imageIds_label_descriptions = pd.merge(df_train_annotations_human_imagelabels_imgIDs_Labels,
                 df_trainables[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [39]:
df_trainables_imageIds_label_descriptions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21214833 entries, 0 to 21214832
Data columns (total 3 columns):
ImageID        object
LabelName      object
Description    object
dtypes: object(3)
memory usage: 647.4+ MB


In [41]:
df_trainables_imageIds_label_descriptions.head(100)

,ImageID,LabelName,Description
0,000002b66c9c498e,/m/01kcnl,Birthday
1,000004f4400f6ec5,/m/01kcnl,Birthday
2,0000071d71a0a6f6,/m/01kcnl,Birthday
3,000007e5752f8f1c,/m/01kcnl,Birthday
4,00001bc2c4027449,/m/01kcnl,Birthday
5,000025ea48cab6fc,/m/01kcnl,Birthday
6,00002b368e91b947,/m/01kcnl,Birthday
7,000060e3121c7305,/m/01kcnl,Birthday
8,000069a0b17c906e,/m/01kcnl,Birthday
9,00006af3eaaad9b6,/m/01kcnl,Birthday


In [42]:
df_trainables_imageIds_label_descriptions.to_csv( data_dir_path + 'imageIDs_LabelNames.csv', sep=',')

## EDA on train-annotations-machine-imagelabels.csv

In [30]:
train_annotations_machine_imagelabels_file_path = data_labels_description_dir_path + 'train-annotations-machine-imagelabels.csv'

In [ ]:
df_train_annotations_machine_imagelabels = pd.read_csv(train_annotations_machine_imagelabels_file_path,header=0)

In [ ]:
df_train_annotations_machine_imagelabels.head()

In [ ]:
## EDA on train-annotations-human-imagelabels.csv